# TWITTER SENTIMENT ANALYSIS

Steps Involved
1. Data exploration.
2. Data Cleaning
3. Moddeling
4. Performance Analysis

# DATA EXPLORATION AND CLEANING

Improting Dependencies

In [1]:
import pandas as pd
import nltk
df_Raw_tweets = pd.read_csv(r'C:\Users\nkhg\Desktop\AIML\30 Projects\DAY2\Tweets.csv')

In [2]:
print(df_Raw_tweets.shape)
df_Raw_tweets.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
# Removing airline_sentiment_confidence which is less than 40 percent
df_Raw_tweets = df_Raw_tweets.drop(df_Raw_tweets[df_Raw_tweets['airline_sentiment_confidence']<.8].index, axis =0)
df_Raw_tweets.shape

(10459, 15)

In [4]:
x_feature = df_Raw_tweets['text']
y_labels = df_Raw_tweets['airline_sentiment']

# CLEANING TEXT DATA

In [5]:
# Removing stop words,punctuation,numbers.
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nkhg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stopword =  stopwords.words('english')
punct = string.punctuation
stemmer = PorterStemmer()

In [7]:
import re
clean_data = [] #creating an empty list
for i in range(len(x_feature)):
    tweet = re.sub('[^a-zA-Z]',' ',x_feature.iloc[i])
    tweet = tweet.lower().split()
    tweet = [stemmer.stem(word) for word in tweet if (word not in stopword) and (word not in punct) and (word != 'virginamerica')]
    tweet = ' '.join(tweet)
    clean_data.append(tweet)

In [8]:
#scentence 
clean_data[1000:1005]

['unit stuck ogg look like flight cancel flightl help',
 'unit yea book next aval flight sinc sat time come plane nd day work miss hotelliv',
 'unit current cri want ski tomorrow prolli boy tryna fulfil dream',
 'unit tri onlin track said someth like attempt locat luggag',
 'unit worker told us call talk someon thailand']

In [9]:
# Encoding catogorical data
y_labels.value_counts()

negative    7392
neutral     1550
positive    1517
Name: airline_sentiment, dtype: int64

In [10]:
y_labels = y_labels.astype('category')
y_labels = y_labels.cat.codes
y_labels.value_counts()

0    7392
1    1550
2    1517
dtype: int64

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
countvextor = CountVectorizer(max_features=3000,stop_words=['unit'])
bag_of_words = countvextor.fit_transform(clean_data).toarray()
bag_of_words.shape

(10459, 3000)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model = MultinomialNB()
X_train,X_test,y_train,y_test=train_test_split(bag_of_words,y_labels,test_size=0.3)



In [13]:
model.fit(X_train,y_train)

MultinomialNB()

In [14]:
y_predict = model.predict(X_test)

In [15]:
from sklearn.metrics import classification_report
Classification_report = classification_report(y_test,y_predict)

In [16]:
print(Classification_report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      2219
           1       0.63      0.46      0.53       478
           2       0.76      0.75      0.75       441

    accuracy                           0.83      3138
   macro avg       0.75      0.71      0.73      3138
weighted avg       0.82      0.83      0.82      3138

